# Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import torch
import scipy.sparse as sp
import torch.nn as nn
from node2vec import Node2Vec
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import itertools
import dgl

/home/pintos/miniconda3/envs/DataScience/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import the data
# df = pd.read_csv('../RVF_ATX_PID_HZ-2020-07.tsv', sep='\t')
df = pd.read_csv('../RVF_ATX_PID_HZ-2020-07.tsv', sep='\t')
# filter out certain columns
df = df.drop(columns=['local_date', 'local_hour', 'gender', 'age', 'full_panel_reweighted_sag_score', 'home_zip'])
dwell_time_threshold = 60
df = df[df['dwell'] >= dwell_time_threshold]
df.head()

,persistentid,venueid,utc_date,utc_hour,dwell
0,5903d26cdcecbd13590c8fe594de785f19b16004e19156...,40b52f80f964a52051001fe3,2020-07-18,22,60
1,cc371dcb888f9ec0ab9a4ecabc4d49e51288313fb17a47...,40b52f80f964a52051001fe3,2020-07-18,22,1011
2,21e09ed692d56697e3c26b777a53a411cd21bed5527c5b...,40b52f80f964a52051001fe3,2020-07-25,0,1058
4,63164c43b459b4260a338979948ead113855ddaafb9d52...,40b52f80f964a52051001fe3,2020-07-29,18,763
5,ad9adea8e7d63428e9372e0d670244e5033d4d2988e554...,40b52f80f964a52051001fe3,2020-07-03,20,9208


In [6]:
# def add_edges_within_group(group, network):
#     # group is a dataframe
#     # G is a graph
#     # add edges between all pairs of nodes in the group
#     num_rows = group.shape[0]
#     if num_rows == 1:
#         return
#     for i in range(num_rows):
#         for j in range(i+1, num_rows):
#             # node1 = group.iloc[i]['persistentid']
#             # node2 = group.iloc[j]['persistentid']

#             node1 = group['persistentid'].iloc[i]
#             node2 = group['persistentid'].iloc[j]

#             # if G.has_edge(node1, node2):
#             #     G[node1][node2]['weight'] += 1
#             # else:
#             #     G.add_edge(node1, node2, weight=1)
#             if node1 != node2:
#                 # G.add_edge(node1, node2)
#                 network.loc[len(network)] = [node1, node2]
            
def add_edges_within_group(group, G):
    # group is a dataframe
    # G is a graph
    # add edges between all pairs of nodes in the group
    num_rows = group.shape[0]
    if num_rows == 1:
        return
    for i in range(num_rows):
        for j in range(i+1, num_rows):
            node1 = group.iloc[i]['persistentid']
            node2 = group.iloc[j]['persistentid']
            # if G.has_edge(node1, node2):
            #     G[node1][node2]['weight'] += 1
            # else:
            #     G.add_edge(node1, node2, weight=1)
            if node1 != node2:
                G.add_edge(node1, node2)

# date = '2020-07-' + str(i+1).zfill(2)
start_date = '2020-07-01'
end_date = '2020-07-05'
# print(date)

# filter the data for date range
# df_date_filtered = df[df['utc_date'] == date]
df_date_filtered = df[(pd.to_datetime(df['utc_date']) >= start_date) & (pd.to_datetime(df['utc_date']) <= end_date)]
# print(len(df_date_filtered))

# group the data by venue_id
df_date_filtered_grouped_by_venue = df_date_filtered.groupby(['venueid', 'utc_hour', 'utc_date'])

# network = pd.DataFrame(columns=['node1', 'node2'])
# df_date_filtered_grouped_by_venue.apply(add_edges_within_group, network)
# G = nx.from_pandas_edgelist(network, 'node1', 'node2')


G = nx.Graph()
df_date_filtered_grouped_by_venue.apply(add_edges_within_group, G=G)

# save the graph G as a .gexf file
target_file_location = f"5_Day_Network_2.gexf"
nx.write_gexf(G, target_file_location)

